In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.optimizers import Adam

import os
from utils.eda_functions import (load_from_pickle, save_to_pickle)
from config import (DATA_DIR)

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
# Define the GRU model
def build_gru_model(input_shape, units, output_size, learning_rate):
    model = Sequential()
    # Add the GRU layer with specified units and input shape
    model.add(GRU(units=units, input_shape=input_shape, return_sequences=False))
    # Add a Dense layer with softmax activation for classification
    model.add(Dense(output_size, activation='softmax'))

    # Compile the model with the Adam optimizer and categorical crossentropy as the loss function for multi-class classification
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
X_train = load_from_pickle(os.path.join(DATA_DIR, 'X_train.pkl'))
X_dev = load_from_pickle(os.path.join(DATA_DIR, 'X_dev.pkl'))
X_test = load_from_pickle(os.path.join(DATA_DIR, 'X_test.pkl'))

Ys_train = load_from_pickle(os.path.join(DATA_DIR, 'Y_train.pkl'))
Ys_dev = load_from_pickle(os.path.join(DATA_DIR, 'Y_dev.pkl'))
Ys_test = load_from_pickle(os.path.join(DATA_DIR, 'Y_test.pkl'))

In [ ]:
y_train = Ys_train.drop(columns=['mort_icu'])
y_dev = Ys_dev.drop(columns=['mort_icu'])
y_test = Ys_test.drop(columns=['mort_icu'])

In [ ]:
# Assuming X_train is shaped as (number_of_samples, time_steps, features)
input_shape = X_train.shape[1:]  # This should be (time_steps, features)
units = 24  # Number of GRU units
output_size = y_train.shape[1]  # Number of output classes
learning_rate = 0.001  # Learning rate for the optimizer

# Build the model
gru_model = build_gru_model(input_shape, units, output_size, learning_rate)

# Train the model
history = gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_dev, y_dev))

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = gru_model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")